This notebook will demonstrate how to use the **constrained training algorithms** implemented in this toolkit.

To train a network, instantiate an algorithm, passing to it the model, the dataset, a list of `FairnessConstraint`s and the algorithm's hyperparameters.

Load and prepare data from `folktables`:

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from folktables import ACSDataSource, ACSIncome

# load folktables data
data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=["OK"], download=True)
features, labels, groups = ACSIncome.df_to_numpy(acs_data)
# split
X_train, X_test, y_train, y_test, groups_train, groups_test = train_test_split(
    features, labels, groups, test_size=0.2, random_state=42)
# scale
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# make into a pytorch dataset, remove the sensitive attribute (RAC1P)
features_train = torch.tensor(X_train, dtype=torch.float32)[:,:-1]
labels_train = torch.tensor(y_train,dtype=torch.float32)
dataset_train = torch.utils.data.TensorDataset(features_train, labels_train)

c_batch_size = 128
min_subgroup_size = c_batch_size
# For each subgroup, FairnessConstraint needs a list of indices of samples belonging to that subgroup
group_indices_train = [
    np.nonzero(groups_train == group_id)[0] for group_id in np.unique(groups_train)
    if np.count_nonzero(groups == group_id) > min_subgroup_size
]

# repeat for test set
features_test = torch.tensor(X_test, dtype=torch.float32)[:,:-1]
labels_test = torch.tensor(y_test,dtype=torch.float32)
dataset_test = torch.utils.data.TensorDataset(features_test, labels_test)
group_indices_test = [
    np.nonzero(groups_test == group_id)[0] for group_id in np.unique(groups_test)
    if np.count_nonzero(groups == group_id) > min_subgroup_size
]

print(f'Protected attribute: {ACSIncome.group}')
print(f'Number of subgroups considered: {len(group_indices_train)}')
print(f'Size of subgroups: {[len(g) for g in group_indices_train]}')

Let's say we want to add an equal loss constraint on the model.

We can do that by using the `FairnessConstraint` class, which will handle sampling (if possible, sampling an equal number of samples from each relevant subgroup in each minibatch), and passing it the function that will calculate the value of the constraint.

In [ ]:
from itertools import combinations
from src.constraints.constraint import FairnessConstraint
from src.constraints.constraint_fns import abs_loss_equality, tpr_equality

# the protected attribute is "Race" (RAC1P)
# we put a pairwise constraint on each combination of subgroups
constraint_bound = 0.05
constraints = []
for gr1, gr2 in combinations(group_indices_train, 2):
    c = FairnessConstraint(
        dataset=dataset_train,
        group_indices=[gr1, gr2],
        # subtract bound from absolute loss difference to bring constraint to form $c \leq 0$
        fn=lambda model, samples: tpr_equality(torch.nn.BCEWithLogitsLoss(), model, samples) - constraint_bound,
        batch_size=c_batch_size
    )

    constraints.append(c)

print(f'Number of constraints: {len(constraints)}')

In [ ]:
# helper function to analyze model performance

def model_stats(model, features, labels, groups, constraints, constraint_bound):
    with torch.inference_mode():
        gr_ind = list(combinations(groups, 2))
        vals = []
        acc_dif = []
        for i, c in enumerate(constraints):
            idx1, idx2 = gr_ind[i]
            val = c.eval(model, [(features[idx1], labels[idx1]), (features[idx2], labels[idx2])]) + constraint_bound
            vals.append(val.numpy().item())

            logits1 = model(features[idx1])
            logits2 = model(features[idx2])
            outs1 = torch.nn.functional.sigmoid(logits1).numpy()
            outs2 = torch.nn.functional.sigmoid(logits2).numpy()
            preds1 = (outs1.T > 0.5).astype(float)
            preds2 = (outs2.T > 0.5).astype(float) 
            acc1 = np.mean(preds1 == labels[idx1].numpy())
            acc2 = np.mean(preds2 == labels[idx2].numpy())
            acc_dif.append(abs(acc1-acc2))

        print(f'constraints (should be <= {constraint_bound}):')
        print(np.round(vals, decimals=3))
        print(f'c mean: {np.mean(vals)}')
        print(f'c min: {np.min(vals)}')
        print(f'c max: {np.max(vals)}')
        print('---')

        logits = model(features)
        outs = torch.nn.functional.sigmoid(logits).numpy()
        preds = (outs.T > 0.5).astype(float)
        acc = np.sum(preds == labels.numpy())/len(labels)
        print(f'accuracy: {acc}')
        print('accuracy abs. difference:')
        print(np.round(acc_dif, decimals=3))
        print(f'acc abs dif mean: {np.mean(acc_dif)}')
        print(f'acc abs dif min: {np.min(acc_dif)}')
        print(f'acc abs dif max: {np.max(acc_dif)}')

---
---

For comparison, let us first train a model **without constraints**.

Define a model:

In [ ]:
from torch.nn import Sequential
hsize1 = 64
hsize2 = 32
model_uncon = Sequential(
    torch.nn.Linear(features_train.shape[1], hsize1),
    torch.nn.ReLU(),
    torch.nn.Linear(hsize1, hsize2),
    torch.nn.ReLU(),
    torch.nn.Linear(hsize2, 1)
)

And start training:

In [ ]:
from torch.optim import Adam

loader = torch.utils.data.DataLoader(dataset_train, batch_size=32, shuffle=True)
loss = torch.nn.BCEWithLogitsLoss()
optimizer = Adam(model_uncon.parameters())
epochs = 100

for epoch in range(epochs):
    losses = []
    for batch_feat, batch_label in loader:
        optimizer.zero_grad()

        logit = model_uncon(batch_feat)
        loss = torch.nn.functional.binary_cross_entropy_with_logits(logit.squeeze(), batch_label)
        loss.backward()

        optimizer.step()
        losses.append(loss.item())
    print(f"Epoch: {epoch}, loss: {np.mean(losses)}")

Let's now analyze how well the **unconstrained** model does in terms of constraints:

In [ ]:
print('TRAIN')
model_stats(model_uncon, features_train, labels_train, group_indices_train, constraints, constraint_bound)

In [ ]:
print('TEST')
model_stats(model_uncon, features_test, labels_test, group_indices_test, constraints, constraint_bound)

---
---

Now let us train the same model with one of the **constrained** training algorithms:

In [ ]:
from src.algorithms import SSLALM, SSG
from torch.nn import Sequential

hsize1 = 64
hsize2 = 32
model = Sequential(
    torch.nn.Linear(features_train.shape[1], hsize1),
    torch.nn.ReLU(),
    torch.nn.Linear(hsize1, hsize2),
    torch.nn.ReLU(),
    torch.nn.Linear(hsize2, 1)
)

optimizer = SSG(
    net=model,
    data=dataset_train,
    loss=torch.nn.BCEWithLogitsLoss(),
    constraints=constraints,
)

history = optimizer.optimize(
    max_runtime=180,
    batch_size=32,
    seed=42,
    ctol=0.1,
    f_stepsize_rule='dimin',
    f_stepsize=0.1,
    c_stepsize_rule='dimin',
    c_stepsize=0.1
    )

In [ ]:
print('TRAIN')
model_stats(model, features_train, labels_train, group_indices_train, constraints, constraint_bound)

In [ ]:
print('TEST')
model_stats(model, features_test, labels_test, group_indices_test, constraints, constraint_bound)